# 选择数据来源

- 时光网
    - http://movie.mtime.com/262895/details.html#company 
    - 不考虑，数据不全，没有出品方
- 灯塔专业版（淘票票专业版）
    - https://h5.m.taobao.com/app/movie/pages/show-info/index.html?showId=1274916&infoType=relatedCompany
    - 可以考虑，有一定爬取难度
- 猫眼专业版
    - http://piaofang.maoyan.com/movie/1217123
    - 可以考虑，有一定爬取难度
- 百度百科
    - https://baike.baidu.com/item/%E5%A4%BA%E5%86%A0/24272902?fr=aladdin#6
    - 不考虑，不一定每部电影都有相关公司数据

# 要导入的模块

In [1]:
import requests, re
from bs4 import BeautifulSoup

# 全局设置

In [2]:

user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.130 Safari/537.36'

headers = {
    'User-Agent': user_agent
}

session = requests.session()


# 预设两部电影，国内和国外
film_name_china = '夺冠'
film_name_abroad = '阿凡达'

# 猫眼专业版

## 获取具有film_name的电影列表

In [6]:
film_list_url = 'https://maoyan.com/query?kw=' + film_name_china
film_list_res = session.get(url=film_list_url, headers=headers)
film_list_res

<Response [200]>

## 默认取列表第一部电影的ID

In [7]:
film_list_soup = BeautifulSoup(film_list_res.text, 'html.parser')
film_id = film_list_soup.find('div', class_='movie-item').find('a')['href']
film_id = re.search('\d+', film_id).group()
film_id

'1217123'

## 获取参与公司列表

In [8]:
companies_list_url = 'http://piaofang.maoyan.com/movie/' + film_id + '/companylist'
companies_list_res = session.get(url=companies_list_url, headers=headers)
companies_list_res

<Response [200]>

In [9]:
companies_list_soup = BeautifulSoup(companies_list_res.text, 'html.parser')
companies_items = companies_list_soup.find_all('dl', class_='panel-main category')
len(companies_items)

4

In [10]:
film_related_companies_dict = dict()

In [11]:
for companies_item in companies_items:
    key = companies_item.find('span', class_='title-name').text
    value = list()
    companies = companies_item.find_all('p', class_= 'p-item-name')
    for company in companies:
        value.append(company.text)
    film_related_companies_dict[key] =  value
    
film_related_companies_dict

{'出品': ['北京嘉映春天影业有限公司',
  '我们制作有限公司',
  '华夏电影发行有限责任公司',
  '欢喜传媒集团有限公司',
  '阿里巴巴影业（北京）有限公司',
  '联瑞（上海）影业有限公司',
  '中国电影股份有限公司',
  '峨眉电影集团有限公司',
  '浙江影视（集团）有限公司',
  '文投控股股份有限公司',
  '浙江省文化产业投资集团有限公司'],
 '联合出品': ['汉鼎宇佑传媒科技有限公司',
  '中央纪委中国方正出版社',
  '拍拍文化传媒（无锡）有限公司',
  '邵氏兄弟国际影业有限公司',
  '上海三次元影业有限公司',
  '上海拾谷影业有限公司',
  '好孩子制作有限公司',
  '天津猫眼微影文化传媒有限公司',
  '天下一电影制作有限公司',
  '芳草地影视（北京）有限公司',
  '浙江横店影业有限公司',
  '霍尔果斯金逸影业有限公司',
  '无限频率影视有限公司',
  '北京完美影视传媒有限责任公司',
  '天津天影电影制片有限公司',
  '东海电影集团有限公司',
  '耳东文化（北京）有限公司',
  '北京世纪蜘蛛文化传播有限公司',
  '北京歌亚时代投资管理有限公司',
  '天津磨铁娱乐有限公司',
  '上海复逸文化传播有限公司',
  '西咸新区秦汉野马文化产业发展有限公司',
  '盛耀蔚莱（北京）国际文化发展有限公司',
  '嘉映影业有限公司',
  '北京嘉映文化传媒有限公司',
  '喀什嘉映文化传媒有限公司',
  '亚博思文化传媒（北京）有限公司',
  '天津玖焱文化传媒有限公司',
  '北京盛煜天桐文化传播有限公司',
  '华文映像（北京）影业有限公司',
  '中铭盛世（深圳）影业集团有限公司'],
 '发行': ['上海淘票票影视文化有限公司',
  '厦门联瑞影业有限公司',
  '华夏电影发行有限责任公司',
  '我们亚博思电影发行（北京）有限公司',
  '喀什嘉映文化传媒有限公司',
  '浙江横店影业有限公司',
  '联瑞（上海）影业有限公司',
  '北京新影联影业有限责任公司',
  '上海红星美凯龙影业发展有限公司',
  '广州市光美文化有限公司'],
 '其他': ['薯片文化传播发展（北京）有限责任公司（宣

## 封装

In [12]:
# 输入电影名称
# 输出电影相关企业dict
def get_film_related_companies(film_name):
    film_list_url = 'https://maoyan.com/query?kw=' + film_name
    film_list_res = session.get(url=film_list_url, headers=headers)
    film_list_soup = BeautifulSoup(film_list_res.text, 'html.parser')
    film_id = film_list_soup.find('div', class_='movie-item').find('a')['href']
    film_id = re.search('\d+', film_id).group()
    
    companies_list_url = 'http://piaofang.maoyan.com/movie/' + film_id + '/companylist'
    companies_list_res = session.get(url=companies_list_url, headers=headers)
    companies_list_soup = BeautifulSoup(companies_list_res.text, 'html.parser')
    companies_items = companies_list_soup.find_all('dl', class_='panel-main category')
    
    film_related_companies_dict = dict()
    for companies_item in companies_items:
        key = companies_item.find('span', class_='title-name').text
        value = list()
        companies = companies_item.find_all('p', class_= 'p-item-name')
        for company in companies:
            value.append(company.text)
        film_related_companies_dict[key] =  value
    
    return film_related_companies_dict

In [13]:
get_film_related_companies('囧妈')

{'制作': ['北京真乐道文化传播有限公司'],
 '出品': ['上海欢神喜佑文化有限公司',
  '欢喜传媒集团有限公司',
  '北京真乐道文化传播有限公司',
  '浙江横店影业有限公司',
  '真乐道（上海）影视有限公司',
  '天津猫眼微影文化传媒有限公司',
  '北京欢喜首映文化有限公司',
  '欢欢喜喜（天津）文化投资有限公司',
  '墨客行影业（北京）有限公司',
  '广东昇格传媒股份有限公司'],
 '联合出品': ['中国电影股份有限公司',
  '华夏电影发行有限责任公司',
  '英皇影业有限公司',
  '上海猫眼影业有限公司',
  '耳东影业（北京）有限公司',
  '华文映像（北京）影业有限公司',
  '上海众合千澄影视文化传媒有限公司',
  '天津玖焱文化传媒有限公司',
  '之升（上海）影业有限公司',
  '耀莱影视文化传媒有限公司',
  '北京长城沃美电影院线有限公司',
  '上海真乐道影视文化工作室',
  '中铭盛世（深圳）影业集团有限公司'],
 '发行': ['浙江横店影业有限公司', '北京卓然影业有限公司', '广东昇格传媒股份有限公司'],
 '联合发行': ['中国电影股份有限公司',
  '北京真乐道文化传播有限公司',
  '时代三和文化投资（北京）股份有限公司',
  '北京欢喜首映文化有限公司',
  '华文风泽（北京）影视发行有限公司',
  '北京耀影电影发行有限公司'],
 '其他': ['北京麦特文化发展有限公司（独家营销机构）']}

In [14]:
get_film_related_companies('叶问4')

{'制作': ['天天影画电影制作有限公司（摄制）', '北京丹海咨询有限公司（制作）'],
 '出品': ['上海博纳文化传媒有限公司', '东方影业出品有限公司'],
 '联合出品': ['北京博纳影业集团有限公司',
  '上海天马联合影视文化有限公司',
  '万达影视传媒有限公司',
  '天津猫眼微影文化传媒有限公司',
  '太阳娱乐文化有限公司',
  '耳东文化（北京）有限公司',
  '上海复逸文化传播有限公司',
  '上海鸣肇影视文化有限公司',
  '佛山东天影视文化有限公司',
  '天马影联影视文化（北京）有限公司',
  '博纳影视娱乐有限公司'],
 '发行': ['上海博纳文化传媒有限公司', '天津猫眼微影文化传媒有限公司'],
 '联合发行': ['华夏电影发行有限责任公司', '上海复逸文化传播有限公司'],
 '其他': ['北京蓝天黑马文化传媒有限公司（宣传公司）', '北京六合世纪文化传媒有限公司（视频物料制作）']}

In [15]:
get_film_related_companies('阿凡达')

{'制作': ['TSG Entertainment（联合制作）'],
 '出品': ['美国狮门影片公司', '美国二十世纪福斯电影公司'],
 '联合出品': ['Avatar 2'],
 '发行': ['美国二十世纪福斯电影公司', '二十世纪福斯荷兰分公司', '20世纪福克斯阿根廷分公司', '二十世纪福斯德国分公司']}